In [1]:
with open("input24.txt","r") as f:
    data = f.read().strip().split("\n")

In [2]:
for d in data:
    print(d)

Immune System:
1193 units each with 4200 hit points (immune to slashing, radiation, fire) with an attack that does 33 bludgeoning damage at initiative 2
151 units each with 9047 hit points (immune to slashing, cold; weak to fire) with an attack that does 525 slashing damage at initiative 1
218 units each with 4056 hit points (weak to radiation; immune to fire, slashing) with an attack that does 176 fire damage at initiative 9
5066 units each with 4687 hit points (weak to slashing, fire) with an attack that does 8 slashing damage at initiative 8
2023 units each with 5427 hit points (weak to slashing) with an attack that does 22 slashing damage at initiative 3
3427 units each with 5532 hit points (weak to slashing) with an attack that does 15 cold damage at initiative 13
1524 units each with 8584 hit points (immune to fire) with an attack that does 49 fire damage at initiative 7
82 units each with 2975 hit points (weak to cold, fire) with an attack that does 359 bludgeoning damage at ini

In [3]:
import re

In [4]:
def get_groups(data):
    groups = []

    for r in data:
        army_ = re.findall("Immune|Infection",r)
        if army_:
            army = army_[0]
            i = 1

        #units, hp, damage, initiative
        attr_num = list(map(int,re.findall("\d+",r)))
        attr_str = re.findall("weak|immune|cold|fire|slashing|radiation|bludgeoning",r)
        if attr_num:
            units, hp, attack_damage, initiative = attr_num
            attack_type = attr_str[-1]
            id_ = "{}{}".format(army,i)
            attrs = attr_str[:-1]

            weakI = attr_str.index("weak") if "weak" in attr_str else -1
            immuneI = attr_str.index("immune") if "immune" in attr_str else -1
            attr_dict = {"weak":attr_str[weakI+1:immuneI] if weakI<immuneI else attr_str[weakI+1:-1], 
                         "immune":attr_str[immuneI+1:weakI] if immuneI<weakI else attr_str[immuneI+1:-1]}

            groups.append(Group(army, id_, units, hp, attr_dict, attack_damage, attack_type, initiative))

            i+=1

    return groups

In [5]:
class Group:
    def __init__(self, army, id_, units, hp, attr_dict, attack_damage, attack_type, initiative):
        self.army = army
        self.id_ = id_
        self.units = units
        self.hp = hp
        self.attr_dict = attr_dict
        self.attack_damage = attack_damage
        self.attack_type = attack_type
        self.initiative = initiative
        
        self.attacker = None
        self.attacking = None
        
        self.eff_pow = units * attack_damage
    def __repr__(self):
        return self.id_
    def __str__(self):
        return self.id_

In [6]:
def get_damage(u_attack, u_defend):
    base_damage = u_attack.attack_damage * u_attack.units
    if u_attack.attack_type in u_defend.attr_dict["immune"]:
        base_damage = 0
    elif u_attack.attack_type in u_defend.attr_dict["weak"]:
        base_damage*=2
    return base_damage

In [7]:
def get_deaths(g, damage):
    deaths = damage // g.hp
    return deaths

In [19]:
def get_enemy(possible):
    #Keep max damage
    possible = [x for x in possible if x[1]==max([y[1] for y in possible])]
    #If tie - enemy with largest effective power
    if len(possible)>1:
        possible = [x for x in possible if x[0].eff_pow==max([y[0].eff_pow for y in possible])]
    #If still tie - defending group with highest initiative
    if len(possible)>1:
        possible = [max(possible,key=lambda x:x[0].initiative)]
    
    return possible[0]

In [20]:
def run_sim(groups, immune_must_win=False):
    while True:
        groups = [x for x in groups if x.units>0]
        groups.sort(key=lambda x:x.initiative, reverse=True)
        groups.sort(key=lambda x:x.eff_pow, reverse=True)

        infection = [x for x in groups if x.army=="Infection"]
        immune = [x for x in groups if x.army=="Immune"]
        
        #Condition for part 2
        if immune_must_win and not immune:
            return 0        
        if not infection or not immune:
            break

        for g in groups:
            enemies = [x for x in groups if x.army!=g.army and not x.attacker]
            enemies = [(x,get_damage(g,x)) for x in enemies]
            possible = [x for x in enemies if x[1]>0]

            if possible:
                #Keep max damage
                possible = [x for x in possible if x[1]==max([y[1] for y in possible])]
                #If tie - enemy with largest effective power
                if len(possible)>1:
                    possible = [x for x in possible if x[0].eff_pow==max([y[0].eff_pow for y in possible])]
                #If still tie - defending group with highest initiative
                if len(possible)>1:
                    possible = [max(possible,key=lambda x:x[0].initiative)]
                
                assert len(possible)==1
                possible[0][0].attacker = g
                g.attacking = possible[0][0]

        
        #Sort by initiative, then attack
        groups.sort(key=lambda x:x.initiative, reverse=True) 
        total_deaths = 0
        for g in groups:
            if g.attacking:
                damage = get_damage(g, g.attacking)
                deaths = damage // g.attacking.hp
                total_deaths+=deaths
                remaining_units = max(g.attacking.units - deaths, 0)
                
                g.attacking.units = remaining_units

        #Can get stuck in a stalemate
        if total_deaths==0:
            return 0
        
        #Update values for next iteration
        for g in groups:
            g.attacker = None
            g.attacking = None
            g.eff_pow = g.units * g.attack_damage

    return sum([g.units for g in groups])


In [21]:
groups = get_groups(data)
print(run_sim(groups))

28976


In [22]:
win_total = 0
increase = 0
while not win_total:
    increase+=1
    
    groups = get_groups(data)
    for g in groups:
        if g.army=="Immune":
            g.attack_damage+=increase
            g.eff_pow = g.attack_damage*g.units

    win_total = run_sim(groups, True)
print(win_total)

3534
